In [192]:
import hid
import time

VID = 0x0403
PID = 0x6030
INTERFACE = 1

In [193]:
# Tady se nalezne spravny HID zarizeni dle VID/PID a vybere se UART interface

devices = hid.enumerate()

for d in devices:
    if d['vendor_id'] == VID and d['product_id'] == PID:
        print("Nalezeno zařízení:", d)

        if d['interface_number'] == INTERFACE:
            try:
                device = hid.device()
                device.open_path(d['path'])
            finally:
                print("Vybrane zarizeni", device)
                #device.close()

Nalezeno zařízení: {'path': b'5-2.2:1.0', 'vendor_id': 1027, 'product_id': 24624, 'serial_number': '', 'release_number': 8704, 'manufacturer_string': 'FTDI', 'product_string': 'FT260', 'usage_page': 0, 'usage': 0, 'interface_number': 0}
Nalezeno zařízení: {'path': b'5-2.2:1.1', 'vendor_id': 1027, 'product_id': 24624, 'serial_number': '', 'release_number': 8704, 'manufacturer_string': 'FTDI', 'product_string': 'FT260', 'usage_page': 0, 'usage': 0, 'interface_number': 1}
Vybrane zarizeni <hid.device object at 0x7f96c43dfbc0>


In [194]:
print(d['path'])

b'5-2.3.5:1.0'


In [188]:
# Otevrit HID zarizeni

#device = hid.device()
#device.open_path(d['path'])

In [190]:
# Tohle přeskočit :) 

device.close()

In [201]:
# Vypis zakladni parametry o zarizeni 
try:
    print(f'Device manufacturer: {device.get_manufacturer_string()}')
    print(f'Product: {device.get_product_string()}')
    print(f'Serial Number: {device.get_serial_number_string()}')

    device.set_nonblocking(1)

except IOError as e:
    print("Chyba při práci s HID zařízením:", e)


Device manufacturer: FTDI
Product: FT260
Serial Number: Љ


In [163]:
# Smycka, ktera vycita RX buffer u FTDI

try:
    while True:
        # Čtení dat
        data = device.read(64)  # Přečte až 64 bajtů dat
        if data:
            print("Přijatá data:", [hex(x) for x in data[2:2+data[1]]], data)
            #device.write(data[2])

        time.sleep(0.05)  # Krátká pauza pro snížení zatížení procesoru

except IOError as e:
    print("Chyba při práci s HID zařízením:", e)
#finally:
#    # Uzavření zařízení
#    #device.close()


ValueError: not open

Tohle je funkcni blok, ktery posle obsah promenne 'zprava' po seriove lince. 

In [168]:
# Poslat UART zpravu do arduina

zprava = b'\n\r Ahoj Testovaci zprava'
l = len(zprava)
payload = bytes([0xFE, l]) + zprava
print(l, payload)
device.write(payload)

24 b'\xfe\x18\n\r Ahoj Testovaci zprava'


26

In [ ]:
device.read(100)

In [202]:
help(device.get_feature_report)

Help on built-in function get_feature_report:

get_feature_report(...) method of hid.device instance
    Receive feature report.
    
    :param report_num:
    :type report_num: int
    :param max_length:
    :type max_length: int
    :return: Incoming feature report
    :rtype: List[int]
    :raises ValueError: If connection is not opened.
    :raises IOError:



In [18]:
d = device.get_feature_report(0xA1, 100)
da = [hex(x) for x in d]
print(len(d))
print(da)

print('flow_ctrl: {}'.format(d[2]))

25
['0xa1', '0x3', '0x2', '0x0', '0x1', '0x1', '0x3', '0x0', '0x1', '0x3', '0x6', '0x1', '0x1', '0xc', '0x1', '0x0', '0x0', '0x0', '0x0', '0x0', '0xac', '0x0', '0xbc', '0x0', '0xcc']
flow_ctrl: 2


In [15]:
# Vypsani aktualniho nastaveni TF260 cipu

d = device.get_feature_report(0xE0, 100)
da = [hex(x) for x in d]
print(len(d))
print(da)
print("")
print('flow_ctrl: {}'.format(d[1]))
print('baud_rade: {}'.format([hex(x) for x in d[2:6]]))
print('data_bit: {}'.format(d[6]))
print('parity: {}'.format(d[7]))
print('stop_bit: {}'.format(d[8]))

10
['0xe0', '0x3', '0xd0', '0x6b', '0x0', '0x0', '0x8', '0x0', '0x0', '0x0']

flow_ctrl: 3
baud_rade: ['0xd0', '0x6b', '0x0', '0x0']
data_bit: 8
parity: 0
stop_bit: 0


In [7]:
device.send_feature_report([0xA1, 0x41, 0x3, 0x00, 0x4B, 0x00, 0x00, 0x08, 0x01, 0x01, 0x00])

-1

# Nastavení baudrate

In [ ]:
# Timto se nastavi baudrate na 9600
device.send_feature_report([0xA1, 0x42, 0x80, 0x25, 0x00, 0x00])

In [157]:
br = 57600
brb = (br).to_bytes(4, byteorder="little")
print(br, brb)
payload = [0xA1, 0x42] + list(brb)
print("payload:", [hex(i) for i in payload])
device.send_feature_report(payload)

57600 b'\x00\xe1\x00\x00'
payload: ['0xa1', '0x42', '0x0', '0xe1', '0x0', '0x0']


6

# Nastavení funkce LEDek

In [172]:
# Vypnuti LEDEk (resp. prepnuti do RX/TX modu)

device.send_feature_report([0xA1, 0x08, 0x04])  # Nastavit GPIOA do TX_LED
device.send_feature_report([0xA1, 0x09, 0x05])  # Nastavit GPIOG do RX_LED

3

# Ovládání DTR pinu, reset arduina

In [21]:
device.send_feature_report([0xB0, 0x00, 0x00, 0b00100000, 0b00100000])
time.sleep(0.1)
device.send_feature_report([0xB0, 0x00, 0x00, 0b00000000, 0b00100000])
#time.sleep(0.1)

5

# Vyčtení dat z UARTu

tvořeno pro programátor, není asi plně funkční...

In [154]:
def read(size=1):
    data_total = []
    for x in range(100):
        data = device.read(100)
        print(x, data)
        if data:
            if len(data):
                print(f"RECV [{size}]< ", [hex(x) for x in data])
                print(f"RECV [{size}]< ", [hex(x) for x in data[2:2+data[1]]])
                data_total += data[2:2+data[1]]
        if len(data_total) >= size:
            print("Done", len(data_total))
            return data_total
        time.sleep(0.05)
    return []
    

In [160]:
print([hex(a) for a in read(30)])

0 [240, 1, 48, 106, 13, 10]
RECV [30]<  ['0xf0', '0x1', '0x30', '0x6a', '0xd', '0xa']
RECV [30]<  ['0x30']
1 [240, 1, 10, 106, 13, 10]
RECV [30]<  ['0xf0', '0x1', '0xa', '0x6a', '0xd', '0xa']
RECV [30]<  ['0xa']
2 [240, 3, 111, 106, 13, 10]
RECV [30]<  ['0xf0', '0x3', '0x6f', '0x6a', '0xd', '0xa']
RECV [30]<  ['0x6f', '0x6a', '0xd']
3 [240, 1, 10, 106, 13, 10]
RECV [30]<  ['0xf0', '0x1', '0xa', '0x6a', '0xd', '0xa']
RECV [30]<  ['0xa']
4 [240, 1, 48, 106, 13, 10]
RECV [30]<  ['0xf0', '0x1', '0x30', '0x6a', '0xd', '0xa']
RECV [30]<  ['0x30']
5 [240, 1, 13, 106, 13, 10]
RECV [30]<  ['0xf0', '0x1', '0xd', '0x6a', '0xd', '0xa']
RECV [30]<  ['0xd']
6 [240, 1, 10, 106, 13, 10]
RECV [30]<  ['0xf0', '0x1', '0xa', '0x6a', '0xd', '0xa']
RECV [30]<  ['0xa']
7 [240, 1, 65, 106, 13, 10]
RECV [30]<  ['0xf0', '0x1', '0x41', '0x6a', '0xd', '0xa']
RECV [30]<  ['0x41']
8 [240, 1, 104, 106, 13, 10]
RECV [30]<  ['0xf0', '0x1', '0x68', '0x6a', '0xd', '0xa']
RECV [30]<  ['0x68']
9 [240, 1, 111, 106, 13, 10]

# Write to UART 
tento blok funguje, pošle po UARTu string.

Vstupní parametr mají být typu `bytes`, např: `b'aaa'`

In [181]:
def write(data):
    #print("SEND> ", [hex(x) for x in data] )
    data = bytes(data)
    l = len(data)
    payload = bytes([0xFE, l]) + data
    print(l, payload)
    device.write(payload)

In [182]:
#reset arduino
device.send_feature_report([0xB0, 0x00, 0x00, 0b00100000, 0b00100000])
time.sleep(0.1)
device.send_feature_report([0xB0, 0x00, 0x00, 0b00000000, 0b00100000])

write(b'\x30\x20')
time.sleep(0.25)
read(10)

SEND>  ['0x30', '0x20']
2 b'\xfe\x020 '
0 [240, 1, 24, 254, 152, 158]
RECV [10]<  ['0xf0', '0x1', '0x18', '0xfe', '0x98', '0x9e']
RECV [10]<  ['0x18']
1 [240, 2, 6, 254, 152, 158]
RECV [10]<  ['0xf0', '0x2', '0x6', '0xfe', '0x98', '0x9e']
RECV [10]<  ['0x6', '0xfe']
2 [240, 1, 6, 254, 152, 158]
RECV [10]<  ['0xf0', '0x1', '0x6', '0xfe', '0x98', '0x9e']
RECV [10]<  ['0x6']
3 [240, 1, 152, 254, 152, 158]
RECV [10]<  ['0xf0', '0x1', '0x98', '0xfe', '0x98', '0x9e']
RECV [10]<  ['0x98']
4 [240, 1, 128, 254, 152, 158]
RECV [10]<  ['0xf0', '0x1', '0x80', '0xfe', '0x98', '0x9e']
RECV [10]<  ['0x80']
5 [241, 5, 158, 254, 158, 152, 158, 128, 152, 152]
RECV [10]<  ['0xf1', '0x5', '0x9e', '0xfe', '0x9e', '0x98', '0x9e', '0x80', '0x98', '0x98']
RECV [10]<  ['0x9e', '0xfe', '0x9e', '0x98', '0x9e']
Done 11


[24, 6, 254, 6, 152, 128, 158, 254, 158, 152, 158]